# Classification of fake news

I am using the dataset from Kaggle. I first want to retrieve this and convert it to a dataframe object

In [68]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [85]:
# Importer det første datasæt
df1 = pd.read_csv("True.csv")
df2 = pd.read_csv("Fake.csv")

df = pd.concat([df1, df2], ignore_index=True)

X1 = np.array(df["text"])

# Lav target array, hvor 0 er en troværdig artikel og 1 er en utroværdig artikel
a = np.zeros(len(df1))
b = np.ones(len(df2))
y1 = np.hstack((a, b))

# Importer det andet datasæt
df_second = pd.read_csv("Train.csv")
X2 = np.array(df_second["text"])

y2 = np.array(df_second["label"])

# Saml datasættene til et stort datasæt
X = np.append(X1, X2)
y = np.hstack((y1, y2))
print(np.size(y))

65698


In [87]:
import re
from nltk.stem import PorterStemmer
ps = PorterStemmer()
failed_files = []
corpus = []
for i, text in enumerate(X):
    if i % 500 == 0:
        print(i)
    # try-statement benyttes, da nogen artikler blot indeholder nan, hvilket vil give fejl senere
    try:
        text_oa = re.sub("[^a-zA-Z]", " ", text)

        text_oa = text_oa.lower()
        text_oa = text_oa.split()

        # stopwords filtreres fra og artikler med 5 eller færrere ord tilbage fjernes, da vi ikke mener at de bidrager
        # til opfattelsen af fake news. Dette hjælper også til at undgå fejl senere

        filtered_article = [ps.stem(w) for w in text_oa if w not in stop_words and len(w) > 1]
        if len(filtered_article) > 5:
            corpus.append(" ".join(filtered_article))
        else:
            failed_files.append(i)
    except TypeError:
        failed_files.append(i)

# Tilpas target, da nogen artikler er blevet fjernet
print(np.shape(y))
y = np.delete(y, failed_files)
print(np.shape(y))

print(len(X))
print(len(corpus))

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
54500
55000
55500
56000
56500
57000
57500
58000
58500
59000
59500
60000
60500
61000
61500
62000
62500
63000
63500
64000
64500
65000
65500
(65698,)
(64539,)
65698
64539


In [88]:
print(corpus[:5])

['washington reuter head conserv republican faction congress vote month huge expans nation debt pay tax cut call fiscal conserv sunday urg budget restraint keep sharp pivot way among republican repres mark meadow speak cb face nation drew hard line feder spend lawmak brace battl januari return holiday wednesday lawmak begin tri pass feder budget fight like link issu immigr polici even novemb congression elect campaign approach republican seek keep control congress presid donald trump republican want big budget increas militari spend democrat also want proport increas non defens discretionari spend program support educ scientif research infrastructur public health environment protect trump administr alreadi will say go increas non defens discretionari spend percent meadow chairman small influenti hous freedom caucu said program democrat say enough need give govern pay rais percent fiscal conserv see rational eventu run peopl money said meadow among republican vote late decemb parti debt

In [89]:
# Corpus for datasættet omdannes til en bag of words
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(corpus)

print("Vocabulary size: {}".format(len(vect.vocabulary_)))

bag_of_words = vect.transform(corpus)
print("bag_of_words: {}".format(repr(bag_of_words)))

Vocabulary size: 157735
bag_of_words: <64539x157735 sparse matrix of type '<class 'numpy.int64'>'
	with 11869003 stored elements in Compressed Sparse Row format>


In [95]:
# Undersøg parametre for bag_of_words
feature_names = vect.get_feature_names()
print("Number of features: {}".format(len(feature_names)))
print("Every 500th feature:\n{}".format(feature_names[::1000]))

Number of features: 157735
Every 500th feature:
['aa', 'acheson', 'afir', 'akiba', 'allnewspipelin', 'amriki', 'antibiot', 'arb', 'ashkenazim', 'augtellez', 'azzad', 'bankhead', 'bazhina', 'benbarto', 'bfwogsdli', 'blaaaaaagh', 'boghossian', 'boyfriend', 'brocksmith', 'buress', 'callerbut', 'carozza', 'cemment', 'cheater', 'christiebooktitl', 'claybrook', 'colickey', 'conflit', 'cornczech', 'cremitari', 'cursor', 'dannyzuk', 'decirlo', 'deodoro', 'dhati', 'disgfsb', 'dominski', 'dron', 'dystrophin', 'ehgpcnwpg', 'embl', 'enviada', 'esquadra', 'executiveth', 'fallist', 'feltman', 'fisherhil', 'foray', 'fremmed', 'gabl', 'gbmnyc', 'getadslength', 'glaciologist', 'gopwithh', 'grigoropoulo', 'gurkentrupp', 'hambley', 'haussmanian', 'henni', 'hinf', 'hongcai', 'humblot', 'ideologicznej', 'impract', 'inni', 'iordan', 'iwmqedylyxhiem', 'jazirah', 'jinx', 'juc', 'kalvin', 'kbed', 'khazar', 'kleider', 'koryto', 'kuran', 'landofthefre', 'leavi', 'lgm', 'littlelivesmatt', 'losgoy', 'lydic', 'mais

In [91]:
# Dataen opdeles nu i en træningsdel og en testdel
from sklearn.model_selection import train_test_split

# Datasæt 1
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, y, random_state=0)
print("Training articles: {}\nTest articles: {}".format(np.size(y_train), np.size(y_test)))

Training articles: 48404
Test articles: 16135


In [92]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=1000).fit(X_train, y_train)
print("Dataset 1 training score: {}".format(log_reg.score(X_train, y_train)))
print("Dataset 1 test score: {}".format(log_reg.score(X_test, y_test)))

Dataset 1 training score: 0.9997727460540451
Dataset 1 test score: 0.9625658506352649


In [93]:
# Jeg tester modellens generaliseringspotentiale med cross-validation på datasættet.
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, bag_of_words, y, cv=5)
print("Average cross-validation score: {:.2f}".format(scores.mean()))

Average cross-validation score: 0.95
